In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [3]:
import re
import pytest
import joblib
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import bayesian_search_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.preprocessing import RollingFeatures
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
import os

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
THIS_DIR = Path("C:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast\\skforecast\\model_selection\\tests")
THIS_DIR = Path(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests")
series_item_sales = pd.read_parquet(THIS_DIR/'fixture_multi_series_items_sales.parquet')
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [4]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
# from skopt.space import Real
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection._search import bayesian_search_forecaster
from skforecast.model_selection._split import TimeSeriesFold

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y

In [8]:
import os
import re
import sys
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._search import _bayesian_search_optuna_multiseries
from skforecast.model_selection._search import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod

import re
import pytest
import os
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection._search import _bayesian_search_optuna
from skforecast.model_selection._split import TimeSeriesFold
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod

In [17]:
window_features = RollingFeatures(
    stats=['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes=3,
)
regressor = LGBMRegressor(
    n_estimators=20, random_state=123, verbose=-1
)
forecaster = ForecasterRecursiveMultiSeries(
                    regressor          = regressor,
                    lags               = 2, 
                    window_features    = window_features,
                    encoding           = 'onehot',
                    transformer_series = None
                )
cv = TimeSeriesFold(
        initial_train_size = len(series) - 12,
        steps              = 3,
        gap                = 0,
        refit              = False,
        fixed_train_size   = False,
        )
lags_grid = [2, 4]
param_grid = [
    {'n_estimators': 15, 'max_depth': 5},
    {'n_estimators': 10, 'max_depth': 3}
]

results = _evaluate_grid_hyperparameters_multiseries(
                forecaster         = forecaster,
                series             = series_dict,
                exog               = exog_dict,
                param_grid         = param_grid,
                cv                 = cv,
                metric             = 'mean_absolute_error',
                aggregate_metric   = 'weighted_average',
                levels             = None,
                lags_grid          = lags_grid,
                return_best        = False,
                verbose            = False
            )

lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
results.to_dict(orient='list')

{'levels': [['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
  ['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
  ['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
  ['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004']],
 'lags': [array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2])],
 'lags_label': [array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2])],
 'params': [{'n_estimators': 15, 'max_depth': 5},
  {'n_estimators': 15, 'max_depth': 5},
  {'n_estimators': 10, 'max_depth': 3},
  {'n_estimators': 10, 'max_depth': 3}],
 'mean_absolute_error__weighted_average': [663.557949349482,
  673.8431369197542,
  761.8926543280367,
  775.090407239743],
 'n_estimators': [15, 15, 10, 10],
 'max_depth': [5, 5, 3, 3]}

In [ ]:
{'levels': [['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2'],
  ['l1', 'l2']],
 'lags': [np.array([1, 2, 3, 4]),
  np.array([1, 2, 3, 4]),
  np.array([1, 2]),
  np.array([1, 2]),
  np.array([1, 2, 3, 4]),
  np.array([1, 2])],
 'lags_label': [np.array([1, 2, 3, 4]),
  np.array([1, 2, 3, 4]),
  np.array([1, 2]),
  np.array([1, 2]),
  np.array([1, 2, 3, 4]),
  np.array([1, 2])],
 'params': [{'alpha': 1},
  {'alpha': 0.1},
  {'alpha': 1},
  {'alpha': 0.1},
  {'alpha': 0.01},
  {'alpha': 0.01}],
 'mean_absolute_error__weighted_average': [0.2146029752649913,
  0.2155171009179464,
  0.21574815856300625,
  0.21677673611895729,
  0.22004258621311923,
  0.22107598684517502],
 'alpha': [1.0, 0.1, 1.0, 0.1, 0.01, 0.01]}